In [1]:
# Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
# Import data
df = pd.read_csv('/Users/ellyatwood/Documents/GEO490/geospatial-data-science/data/lab5/seattle_house_prices.csv')


In [9]:
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat']))
gdf = gdf.set_crs(4326, allow_override=True)

# Reproject everything to UTM 10N (EPSG:32610)
gdf_utm = gdf.to_crs('EPSG:32610')

In [12]:
# Looking for rows and columns count
print (df.count)

<bound method DataFrame.count of          price  bedrooms  bathrooms  sqft_living  sqft_lot  yr_built      lat  \
0       538000         3       2.25         2570      7242      1951  47.7210   
1       180000         2       1.00          770     10000      1933  47.7379   
2       604000         4       3.00         1960      5000      1965  47.5208   
3       510000         3       2.00         1680      8080      1987  47.6168   
4      1230000         4       4.50         5420    101930      2001  47.6561   
...        ...       ...        ...          ...       ...       ...      ...   
19446   475000         3       2.50         1310      1294      2008  47.5773   
19447   360000         3       2.50         1530      1131      2009  47.6993   
19448   400000         4       2.50         2310      5813      2014  47.5107   
19449   400000         3       2.50         1600      2388      2004  47.5345   
19450   325000         2       0.75         1020      1076      2008  47.594

There are 19451 total houses in the dataset, and 9 features for determining house price

In [14]:
# Looking for null values
print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19451 entries, 0 to 19450
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   price        19451 non-null  int64   
 1   bedrooms     19451 non-null  int64   
 2   bathrooms    19451 non-null  float64 
 3   sqft_living  19451 non-null  int64   
 4   sqft_lot     19451 non-null  int64   
 5   yr_built     19451 non-null  int64   
 6   lat          19451 non-null  float64 
 7   long         19451 non-null  float64 
 8   geometry     19451 non-null  geometry
dtypes: float64(3), geometry(1), int64(5)
memory usage: 1.3 MB
None


In [34]:
# Compute correlation matrix
corr_matrix = gdf_utm.corr()

# Display just house value correlations
corr_matrix["price"].sort_values(ascending= False)

price                   1.000000
sqft_living             0.702296
bathrooms               0.524395
bedrooms                0.315804
lat                     0.308082
living_per_lot          0.126975
sqft_lot                0.090125
yr_built                0.052453
long                    0.020092
bedrooms_per_lot       -0.039601
age                    -0.052453
bathrooms_per_living   -0.267342
bedrooms_per_living    -0.479228
Name: price, dtype: float64

The intial variables that are best correlated with house price is longitude, yr_built, and sqft_lot.
The intial variables that are least correlated with house price is sqft_living, bathrooms, and bedrooms.

In [28]:
# Bedrooms per lot
gdf_utm['bedrooms_per_lot'] = gdf_utm['bedrooms'] / gdf_utm['sqft_lot']

# Bedrooms per living
gdf_utm['bedrooms_per_living'] = gdf_utm['bedrooms'] / gdf_utm['sqft_living']

# Bathrooms per living
gdf_utm['bathrooms_per_living'] = gdf_utm['bathrooms'] / gdf_utm['sqft_living']

# Sqftliving per lot
gdf_utm['living_per_lot'] = gdf_utm['sqft_living'] / gdf_utm['sqft_lot']

# Age
gdf_utm['age'] = 2022 - gdf_utm['yr_built']

In [29]:
# Compute correlation matrix
corr_matrix = gdf_utm.corr()

# Display just house value correlations
corr_matrix["price"].sort_values(ascending= False)

price                   1.000000
sqft_living             0.702296
bathrooms               0.524395
bedrooms                0.315804
lat                     0.308082
living_per_lot          0.126975
sqft_lot                0.090125
yr_built                0.052453
long                    0.020092
bedrooms_per_lot       -0.039601
age                    -0.052453
bathrooms_per_living   -0.267342
bedrooms_per_living    -0.479228
Name: price, dtype: float64

In [30]:
# Define feature list
feature_list =  ['bedrooms', 'bathrooms', 'sqft_living', 
                 'sqft_lot', 'yr_built', 'lat', 'long', 'bedrooms_per_lot', 
                 'bedrooms_per_living', 'bathrooms_per_living', 'living_per_lot',
                'age']

# Define features and labels 
X = gdf_utm[feature_list]
y = gdf_utm['price']

# Standarize data
scaler = StandardScaler()  
X_scaled = scaler.fit_transform(X)



In [31]:
# Split data 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [32]:
# Define model
forest_reg = RandomForestRegressor(n_estimators = 30)

# Fit model
forest_reg.fit(X_train, y_train)

RandomForestRegressor(n_estimators=30)

In [33]:
# Predict test labels predictions
predictions = forest_reg.predict(X_test)

# Compute mean-squared-error
final_mse = mean_squared_error(y_test , predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

160774.30471727348